In [1]:
from EvGym.charge_world import ChargeWorldEnv, Session
from EvGym.charge_agent import agentRB
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from IPython.display import clear_output

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [2]:
df_data = pd.read_csv("data/prepared_elaad_transactions.csv", parse_dates=["TransactionStartDT", "TransactionStopDT"])
df_data["TransactionStartDate"] = pd.to_datetime(df_data["TransactionStartDate"]).dt.date
df_data["TransactionStopDate"] = pd.to_datetime(df_data["TransactionStopDate"]).dt.date
print(f"TransactionStartTS:\t min {df_data['TransactionStartTS'].min()},\tmax {df_data['TransactionStartTS'].max()}")
print(f"TransactionStopTS:\t min {df_data['TransactionStopTS'].min()},\tmax {df_data['TransactionStopTS'].max()}")
print("--------")
df_data.info()

TransactionStartTS:	 min 1.0,	max 35028.0
TransactionStopTS:	 min 33.0,	max 35104.0
--------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionId            10000 non-null  int64         
 1   ChargePoint              10000 non-null  object        
 2   Connector                10000 non-null  int64         
 3   TransactionStartDT       10000 non-null  datetime64[ns]
 4   TransactionStopDT        10000 non-null  datetime64[ns]
 5   StartCard                10000 non-null  object        
 6   ConnectedTime            10000 non-null  float64       
 7   ChargeTime               10000 non-null  float64       
 8   TotalEnergy              10000 non-null  float64       
 9   MaxPower                 10000 non-null  float64       
 10  BatteryCapacity          10000 non-null  int64         
 11  S

In [3]:
df_data[df_data["TransactionStartTS"] == 1].sort_values(["TransactionStartTS"]).head()

,TransactionId,ChargePoint,Connector,TransactionStartDT,TransactionStopDT,StartCard,ConnectedTime,ChargeTime,TotalEnergy,MaxPower,...,TransactionStopSecDay,TransactionStartHourDay,TransactionStopHourDay,TransactionStartAbsSec,TransactionStopAbsSec,TransactionStartTS,TransactionStopTS,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime
0,3261657,e62c50d1be0a2f80ec51d471f9630a4e,2,2019-01-01 00:30:00,2019-01-01 08:24:00,826d337c1d84900452edfcb27a3fb11bcbd06f35da8406abd9ca28b1291d219c,7.91,1.0,6.53,9.818,...,30240.0,0.0,8.0,0.0,28440.0,1.0,33.0,474.6,474.0,0.6


In [4]:
world = ChargeWorldEnv(df_data)
agent = agentRB()


In [ ]:
df_state = world.reset()
df_state_minus1 = df_state.copy()
for i in range(df_data.shape[0]):
    clear_output(wait=True)  # Clear the output from the previous iteration
    action = agent.get_action(df_state)
    df_state, reward, done, info = world.step(action)
    
    display(df_state)  # Display the data frame
    
    #display(f"t= {world.t}, action {action}, power {world.power_t.sum()}")  # Display the iteration number
    
    if world.df_depart.shape[0] > 0:
        for _, dep in world.df_depart.iterrows():
            print(f"{dep.idSess} departed with soc of {dep.soc_t}")
    print(f"t= {world.t}, reward {reward},\npower {world.power_t.sum():.02f}, \naction {action[:10]}\n{action[10:]}")
    
    if (df_state_minus1.values == df_state.values).all():
        time.sleep(0.2)
    else:
        input()
    df_state_minus1 = df_state.copy()


    

,idSess,t_arr,soc_arr,t_dep,soc_t
0,3263694,166,0.086875,223,1
1,-1,0,0,0,0
2,3263398,151,0.08875,238,1
3,3263552,159,0.57825,223,1
4,3263419,152,0.06025,223,1
5,3263753,170,0.308625,215,1
6,3263781,173,0.7015,214,1
7,3263617,162,0.078125,219,1
8,-1,0,0,0,0
9,-1,0,0,0,0


3263607 departed with soc of 1
t= 213, reward 0,
power 0.00, 
action [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000]


In [ ]:
world.reset()
# Before 50-60 sec
for i in range(10): #tqdm(range(df_data["TransactionStopTS"].max().astype(int))):
    world.step(np.zeros(world.max_cars))

#world.step()
#print(world.t)
#world.df_park.head(200)
